In [1]:
#optiver volatility prediction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/Optiver/train.csv')
test = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/Optiver/test.csv')
order_book = pd.read_parquet('/Users/ericp/OneDrive/Documents/GitHub/Optiver/book_train.parquet/stock_id=0')
trade_book = pd.read_parquet('/Users/ericp/OneDrive/Documents/GitHub/Optiver/trade_train.parquet/stock_id=0')

In [3]:
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [ ]:
test.head()

In [ ]:
stock_id = '0'
book_example = order_book[order_book['time_id'] == 5]
book_example['stock_id'] = stock_id
trade_example = trade_book[trade_book['time_id'] == 5]
trade_example['stock_id'] = stock_id

In [ ]:
book_example.head()

In [ ]:
trade_example.head()

In [ ]:
book_example['wt_avg'] = (book_example['bid_price1'] * book_example['ask_size1'] + book_example['ask_price1'] * book_example['bid_size1'])/ (book_example['bid_size1'] + book_example['ask_size1'])

In [ ]:
f, axs = plt.subplots(1)
f.suptitle('Order Book, Stock = 0, time_id = 5')
axs.plot(book_example['wt_avg'])

In [ ]:
f, axs = plt.subplots(1)
f.suptitle('Trade Book, Stock = 0, time_id = 5')
axs.plot(trade_example['price'])

In [ ]:
#create log diffs
def logDiff(stock_prices):
    return np.log(stock_prices).diff()

In [ ]:
book_example['logDifferences'] = logDiff(book_example['wt_avg'])
book_example = book_example[book_example['logDifferences'].notnull()]
trade_example['logDifferences'] = logDiff(trade_example['price'])
trade_example = trade_example[trade_example['logDifferences'].notnull()]

In [ ]:
f, axs = plt.subplots(1)
f.suptitle('LogDiff, OrderBook')
axs.plot(book_example['logDifferences'])

In [ ]:
f, axs = plt.subplots(1)
f.suptitle('LogDiff, TradeBook')
axs.plot(trade_example['logDifferences'])

In [ ]:
#create realized vols for each time / stock price
def realized_vol(log_diffs):
    return np.sqrt(np.sum(log_diffs ** 2))

In [ ]:
print('Realized vol for Stock 0 over Time Period 5 order_book is: {}'.format(realized_vol(book_example['logDifferences'])))
print('Realized vol for Stock 0 over Time Period 5 trade_book is: {}'.format(realized_vol(trade_example['logDifferences'])))

In [ ]:
def RMSPE(vols, truth):         
    return np.sqrt(np.sum(np.mean(np.square((vols - truth)/truth))))   

In [ ]:
orderPath = glob.glob('/Users/ericp/OneDrive/Documents/GitHub/Optiver/book_train.parquet/*')
predictionColumn = 'target'

In [ ]:
def orderBookVol(file, predictionColumn):
    #read file
    order_book = pd.read_parquet(file)
    #create variable for stock_id
    stock_id = file.split('=')[1]
    #create wt avg price
    order_book['wt_avg'] = (order_book['bid_price1'] * order_book['ask_size1'] + 
                            order_book['ask_price1'] * order_book['bid_size1']) / (order_book['bid_size1'] + order_book['ask_size1'])
    #create logDiff
    order_book['logDifferences'] = order_book.groupby(['time_id'])['wt_avg'].apply(logDiff)
    #get rid of NaNs
    order_book = order_book[order_book['logDifferences'].notnull()]
    #create vol estimate
    response = pd.DataFrame(data = order_book.groupby(['time_id'])['logDifferences'].apply(realized_vol)).reset_index()
    response = response.rename(columns = {'logDifferences': predictionColumn})
    #add row_id
    response['row_id'] = response['time_id'].apply(lambda x:f'{stock_id}-{x}')
    
    return response[['row_id', predictionColumn]]
                                                                                       
                                                                                       

In [ ]:
ans = orderBookVol(orderPath[0], predictionColumn)

In [ ]:
#loop thru all files
def loopThruFiles(path, predictionColumn):
    response = pd.DataFrame()
    for file in path:
        answer = orderBookVol(file, predictionColumn)
        response = pd.concat([response, answer], axis = 0)
    return response
        

In [ ]:
naive_model = loopThruFiles(orderPath, predictionColumn)

In [ ]:
naive_model.head()

In [ ]:
naive_model.shape

In [ ]:
naivemodel.to_csv('submission.csv', index = False)